# Niezbalansowana klasyfikacja

![image.png](attachment:image.png)

## Wstęp

Klasyfikacja obrazów to proces przypisywania etykiety do obrazu na podstawie jego zawartości. Przykładowo, chcielibyśmy, aby nasz program komputerowy mógł rozpoznawać, czy na obrazie jest kot, pies, samochód, samolot czy może coś zupełnie innego. W dzisiejszych czasach popularnym narzędziem do rozpoznawania obrazów są tzw. sieci konwolucyjne (CNN).

Sieci konwolucyjne są rodzajem sieci neuronowych, które potrafią analizować i rozpoznawać wzorce w danych wizualnych.

W przypadku klasyfikacji obrazów, sieć konwolucyjna składa się z kilku warstw, w tym konwolucyjnych i poolingowych. Warstwy konwolucyjne służą do ekstrakcji cech z obrazu, następnie za pomocą warstw poolingowych zmniejszamy wymiary danych, a na końcu wykorzystujemy warstwy w pełni połączone do klasyfikacji obrazu.

Progresywne zmniejszanie warstw pozwala sieciom rozpoznawać coraz to bardziej abstrakcyjne cechy jako złożenie wielu pomniejszych cech np. ptak to coś co ma dziób i jest opierzone. Dziób z kolei to np. ostry kształt o żółtawym kolorze a opierzenie oznacza pokrycie dużą ilością małych kresek.

### Zadanie

Zaimplementuj klasyfikator `YourCnnClassifier`, rozpoznający i klasyfikujący obrazki na dwie klasy. Powinna być to konwolucyjna sieć neuronowa napisana z użyciem pakietu `pytorch`.

Twoimi danymi w tym zadaniu są obrazki w formacie \*.jpg o wymiarze 224 x 224. Obrazki te dzielą się na dwie kategorie: *normal* oraz *onion*, którym przypisano odpowiednio etykiety 0 i 1.

Obrazki z klasy *normal* przedstawiają jasnoszare figury na czarnym tle. Natomiast obrazki z klasy *onion* różnią się tym, że mają dodane ciemnoszare pasma tworzące warstwy w środku jasnoszarych figur, co upodabnia je do cebuli. Wszystkie obrazki są dodatkowo zaszumione.

![image-3.png](attachment:image-3.png)
![image-2.png](attachment:image-2.png)

Publicznym interfejsem klasy `YourCnnClassifier` muszą być dwie metody ([class methods](https://stackoverflow.com/questions/12179271/meaning-of-classmethod-and-staticmethod-for-beginner) dokładnie rzecz biorąc):
- `load` - ma wczytać parametry modelu z pliku `cnn-classifier.pth`. Tego będziemy używać podczas testowania twojego rozwiązania
- `create_with_training` - ma wytrenować model i zapisać jego parametry do pliku `cnn-classifier.pth`.

### Kryterium oceny

Twoje rozwiązanie oceniane będzie na podstawie skuteczności klasyfikacji

$$
\mathrm{score}(accuracy) = \begin{cases}
    0 & \text{jeżeli } accuracy < 0.5 \\
    (accuracy - 0.5) * 2 & \text{w.p.p.}
\end{cases}
$$

Powyższe kryterium, klasa abstrakcyjna opisująca interfejs modelu oraz ładowanie danych, są zaimplementowane poniżej przez nas. Jednocześnie podany jest przykład trywialnego klasyfikatora, który zawsze twierdzi, że próbka jest normalna. Tym samym podczas testowania na zbalansowanym zbiorze testowym otrzymuje on 0 pkt.

### Pliki zgłoszeniowe

1. Ten notebook
2. Plik zawierający wagi modelu o nazwie `cnn-classifier.pth`

**Uwaga:** Zbiór danych treningowych, który dostarczamy, jest niezbalansowany, natomiast Twoje rozwiązanie testowane będzie na zbalansowanym zbiorze, aby metryka `accuracy` była miarodajna. Weź to pod uwagę podczas tworzenia swojego modelu.

### Ograniczenia

- Ewaluacja twojego rozwiązania (bez treningu, flaga `FINAL_EVALUATION_MODE` ustawiona na `True`) na 50 przykładach testowych powinna trwać nie dłużej niż 2 minuty na Google Colab **bez** GPU.
- Wykonanie skryptu na Google Colab **bez** GPU z flagą `FINAL_EVALUATION_MODE` ustawioną na `False` powinno wytrenować model i wygenerować plik z wagami w nie więcej niż 15 minut.
- Rozmiar pliku `cnn-classifier.pth` nie powienien przekroczyć 35MB.

## Ewaluacja

Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` możesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających.

Za to zadanie możesz zdobyć pomiędzy 0 i 1 punktów. Liczba punktów, które zdobędziesz będzie równa wartości `score`, wyliczonej na zbiorze testowym.

# Kod startowy

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

import abc
import os

import glob
import gdown
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import zipfile

## Ładowanie danych

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

GDRIVE_DATA = [
    ("1bR87z7ZI3gLK0vAGkyr_cnVGZ9P9bO7A", "train_data.zip"),
    ("1TA0lWnjJCv3lyRMML4JNHsJz3RJ-TUwZ", "valid_data.zip"),
]


def download_data():
    for file_id, zip_name in GDRIVE_DATA:
        folder_name = zip_name.split(".")[0]
        if not os.path.exists(folder_name):
            url = f"https://drive.google.com/uc?id={file_id}"
            gdown.download(url, output=zip_name, quiet=True)
            with zipfile.ZipFile(zip_name, "r") as zip_ref:
                zip_ref.extractall(folder_name)
            os.remove(zip_name)


download_data()


class ImageDataset(torch.utils.data.Dataset):
    """Implementacja abstrakcji zbioru danych z torch'a."""

    def __init__(self, dataset_type: str):
        self.filelist = glob.glob(f"{dataset_type}_data/*")
        self.labels = [0 if "normal" in path else 1 for path in self.filelist]

    def __len__(self):
        return len(self.filelist)

    def __getitem__(self, idx) -> tuple[torch.Tensor, int]:
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image = torchvision.transforms.functional.to_tensor(
            plt.imread(self.filelist[idx])[:, :, 0]
        )
        label = self.labels[idx]
        return image, label

    def loader(self, **kwargs) -> torch.utils.data.DataLoader:
        """
        Stwórz, `DataLoader`'a dla aktualnego zbioru danych.

        Wszystkie `**kwargs` zostaną przekazane do konstruktora `torch.utils.data.DataLoader`.
        `DataLoader`'y w skrócie to abstrakcja ładowania danych usdostępniająca wygodny interfejs.
        Możesz dowiedzieć się o nich więcej tutaj: https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
        """
        return torch.utils.data.DataLoader(self, **kwargs)


train_dataset: ImageDataset = ImageDataset("train")
valid_dataset: ImageDataset = ImageDataset("valid")

## Kod z kryterium oceniającym

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################


def accuracy_to_points(accuracy: float) -> float:
    """Oblicz wynik na podstawie celności predykcji."""
    return (round(accuracy, 2) - 0.5) * 2 if accuracy > 0.5 else 0.0


def grade(model):
    """Oceń ile punktów otrzyma aktualne zadanie."""
    model.eval()
    test_loader = valid_dataset.loader()
    correct = 0
    total = 0
    with torch.no_grad():
        for [images, labels] in test_loader:
            outputs = model(images).squeeze()
            incorrect_indices = torch.where((outputs > 0.5).int() != labels)[0]
            correct += len(labels) - len(incorrect_indices)
            total += len(labels)
        accuracy = correct / total if total != 0 else 0
        if not FINAL_EVALUATION_MODE:
            print(f"Accuracy: {int(round(accuracy, 2) * 100)}%")
        return accuracy_to_points(accuracy)

## Publiczny interfejs rozwiązania

Tylko tego wymagamy od Twojej klasy, w Twoim rozwiązaniu możesz modyfikować swoją klasę do woli dodając nowe metody oraz atrybuty klasy - cokolwiek co będzie Ci potrzebne do rozwiązania zadania.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################


class CnnClassifier(torch.nn.Module, abc.ABC):
    MODEL_PATH: str = "cnn-classifier.pth"

    @classmethod
    def load(cls):
        """Załaduj model z pliku."""
        model = cls()
        model.load_state_dict(torch.load(cls.MODEL_PATH))
        return model

    @classmethod
    @abc.abstractmethod
    def create_with_training(cls):
        """Zapisz model do pliku."""
        pass

## Przykładowe rozwiązanie
Poniżej prezentujemy proste rozwiązanie, które w oczywisty sposób nie jest optymalne. Służy temu, aby było wiadomo w jaki sposób ma działać cały notatnik.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

if not FINAL_EVALUATION_MODE:

    class DummyCnnClassifier(CnnClassifier):
        def forward(self, x):
            batch_size, *_ = x.shape
            return torch.zeros(batch_size)

        @classmethod
        def create_with_training(cls):
            return cls()

    dummy_model = DummyCnnClassifier.create_with_training()
    print(f"DummyCnnClassifier -- Ocena: {grade(dummy_model)} pkt")

    del dummy_model
    del DummyCnnClassifier

# Twoje Rozwiązanie

In [ ]:
class YourCnnClassifier(CnnClassifier):
    """
    Profesjonalna implementacja klasyfikatora CNN dla zadania klasyfikacji niezbalansowanej.

    Ta klasa implementuje sieć konwolucyjną z technikami regularyzacji i obsługą
    niezbalansowanych danych poprzez ważone próbkowanie.
    """

    def __init__(self):
        """
        Inicjalizacja architektury sieci neuronowej.

        Architektura składa się z:
        - Dwóch warstw konwolucyjnych z poolingiem i dropout
        - Warstw w pełni połączonych z regularyzacją
        - Funkcji aktywacji sigmoid na wyjściu (klasyfikacja binarna)
        """
        super().__init__()

        # Definicja architektury sieci jako sekwencja warstw
        self.network = torch.nn.Sequential(
            # Pierwsza warstwa konwolucyjna: 1 kanał wejściowy -> 8 filtrów, kernel 5x5
            torch.nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5),
            torch.nn.ReLU(),  # Funkcja aktywacji ReLU
            torch.nn.MaxPool2d(
                kernel_size=2, stride=2
            ),  # Pooling 2x2 - redukcja wymiarów
            torch.nn.Dropout(p=0.5),  # Dropout 50% - zapobieganie przeuczeniu
            # Druga warstwa konwolucyjna: 8 -> 8 filtrów, kernel 5x5
            torch.nn.Conv2d(in_channels=8, out_channels=8, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(
                kernel_size=4, stride=4
            ),  # Większy pooling - dalsze zmniejszenie
            torch.nn.Dropout(p=0.5),
            # Spłaszczenie tensora do wektora dla warstw w pełni połączonych
            torch.nn.Flatten(),
            # Pierwsza warstwa w pełni połączona: 5408 -> 256 neuronów
            # Rozmiar 5408 wynika z wymiarów po konwolucji i poolingu
            torch.nn.Linear(in_features=5408, out_features=256),
            torch.nn.Dropout(p=0.5),
            torch.nn.ReLU(),
            # Warstwa wyjściowa: 256 -> 1 neuron (klasyfikacja binarna)
            torch.nn.Linear(in_features=256, out_features=1),
            torch.nn.Sigmoid(),  # Sigmoid - wyjście w zakresie [0,1] dla prawdopodobieństwa
        )

    def forward(self, x):
        """
        Propagacja w przód przez sieć.

        Args:
            x: Tensor wejściowy z obrazami [batch_size, channels, height, width]

        Returns:
            Tensor z przewidywaniami [batch_size, 1]
        """
        return self.network(x)

    @classmethod
    def create_with_training(cls):
        """
        Tworzy i trenuje model z obsługą niezbalansowanych danych.

        Implementuje techniki radzenia sobie z niezbalansowanymi klasami:
        1. Obliczanie wag dla każdej klasy odwrotnie proporcjonalnie do częstości
        2. Ważone próbkowanie podczas treningu
        3. Odpowiednia funkcja straty i optymalizator

        Returns:
            Wytrenowany model YourCnnClassifier
        """
        # Inicjalizacja modelu
        model = cls()

        # === ANALIZA ROZKŁADU KLAS ===
        print("Analizowanie rozkładu klas w zbiorze treningowym...")
        cnt_0 = 0  # Licznik dla klasy 0 (klasa większościowa)
        cnt_1 = 0  # Licznik dla klasy 1 (klasa mniejszościowa)

        # Przejście przez cały zbiór treningowy w celu policzenia klas
        for batch_idx, (input_batch, target_batch) in enumerate(train_dataset.loader()):
            for target in target_batch:
                if target.item() == 0:
                    cnt_0 += 1
                else:
                    cnt_1 += 1

        print(f"Klasa 0 (większościowa): {cnt_0} próbek")
        print(f"Klasa 1 (mniejszościowa): {cnt_1} próbek")
        print(f"Stosunek niezbalansowania: {cnt_0/cnt_1:.2f}:1")

        # === OBLICZANIE WAG DLA WAŻONEGO PRÓBKOWANIA ===
        total_samples = cnt_0 + cnt_1
        weights = torch.zeros(total_samples)

        # Wagi odwrotnie proporcjonalne do częstości klas
        # Klasa rzadsza otrzymuje większą wagę
        weight_class_0 = 1.0 / cnt_0  # Mniejsza waga dla klasy większościowej
        weight_class_1 = 1.0 / cnt_1  # Większa waga dla klasy mniejszościowej

        print(f"Waga dla klasy 0: {weight_class_0:.6f}")
        print(f"Waga dla klasy 1: {weight_class_1:.6f}")

        # Przypisanie wag każdej próbce
        sample_idx = 0
        for batch_idx, (input_batch, target_batch) in enumerate(train_dataset.loader()):
            for target in target_batch:
                if target.item() == 0:
                    weights[sample_idx] = weight_class_0
                else:
                    weights[sample_idx] = weight_class_1
                sample_idx += 1

        # === KONFIGURACJA TRENINGU ===
        batch_size = 32
        n_epochs = 20
        learning_rate = 0.001

        # Tworzenie ważonego samplera - zapewnia zbalansowane batche
        total_samples_per_epoch = batch_size * (total_samples // batch_size)
        sampler = torch.utils.data.WeightedRandomSampler(
            weights=weights,
            num_samples=total_samples_per_epoch,
            replacement=True,  # Próbkowanie ze zwracaniem
        )

        # DataLoader z ważonym próbkowaniem
        loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=batch_size, sampler=sampler
        )

        # === KONFIGURACJA OPTYMALIZACJI ===
        # Optymalizator Adam - adaptacyjna szybkość uczenia
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        # Funkcja straty Binary Cross Entropy - odpowiednia dla klasyfikacji binarnej
        criterion = torch.nn.BCELoss(reduction="mean")

        # === PROCES TRENINGU ===
        print(f"\nRozpoczynanie treningu:")
        print(f"- Liczba epok: {n_epochs}")
        print(f"- Rozmiar batcha: {batch_size}")
        print(f"- Szybkość uczenia: {learning_rate}")

        model.train()  # Ustawienie modelu w tryb treningowy

        for epoch in range(n_epochs):
            epoch_loss = 0.0
            batch_count = 0

            for batch_idx, (images, labels) in enumerate(loader):
                # Zerowanie gradientów z poprzedniej iteracji
                optimizer.zero_grad()

                # Propagacja w przód
                predictions = model(images).flatten()  # Spłaszczenie do [batch_size]

                # Obliczenie straty
                loss = criterion(predictions, labels.float())

                # Propagacja wsteczna - obliczenie gradientów
                loss.backward()

                # Aktualizacja wag
                optimizer.step()

                # Akumulacja straty dla statystyk
                epoch_loss += loss.item()
                batch_count += 1

                # Wyświetlanie postępu co kilka batchy
                if batch_idx % 50 == 0:
                    print(
                        f"Epoka {epoch+1}/{n_epochs}, Batch {batch_idx}, "
                        f"Strata: {loss.item():.4f}"
                    )

            # Statystyki na koniec epoki
            avg_epoch_loss = epoch_loss / batch_count
            print(
                f"Epoka {epoch+1}/{n_epochs} zakończona. Średnia strata: {avg_epoch_loss:.4f}"
            )

        # === ZAPISANIE MODELU ===
        print("\nZapisywanie wytrenowanego modelu...")
        torch.save(model.state_dict(), cls.MODEL_PATH)
        print(f"Model zapisany w: {cls.MODEL_PATH}")

        return model

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

your_model = (
    YourCnnClassifier.load()
    if FINAL_EVALUATION_MODE
    else YourCnnClassifier.create_with_training()
)

# Ewaluacja

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################


def evaluate_model(model):
    """Oceń ile punktów otrzyma aktualne zadanie."""
    return grade(model)

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

if not FINAL_EVALUATION_MODE:
    print(f"YourCnnClassifier -- Ocena: {evaluate_model(your_model):.2f} pkt")